In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import numpy as np
import matplotlib as plt
import cv2
import math
import os
import keras
from keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Flatten, Reshape, Activation, Dense, UpSampling2D, LeakyReLU, Input
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import backend as K

Using TensorFlow backend.


In [2]:
images=list()
for i in range(1,5000):
    a=cv2.imread("img_align_celeba/%06d.jpg"%i,1)
    a=cv2.resize(a,(a.shape[0]//2-1,a.shape[0]//2-1))
    a=a/255.0
    a=a.tolist()
    images.append(a)
images=np.asarray(images)
np.save('data.npy',images)

In [3]:
np.load('data.npy')

array([[[[0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         ...,
         [0.79607843, 0.92941176, 1.        ],
         [0.85098039, 0.89803922, 0.96862745],
         [0.87058824, 0.93333333, 0.99607843]],

        [[0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         ...,
         [0.81960784, 0.94117647, 0.99607843],
         [0.86666667, 0.91372549, 0.98431373],
         [0.8745098 , 0.9372549 , 1.        ]],

        [[0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         [0.76078431, 0.90588235, 0.99215686],
         ...,
         [0.81568627, 0.9254902 , 0.98431373],
         [0.87843137, 0.93333333, 0.99607843],
         [0.86666667, 0.92941176, 0.99215686]],

        ...,

        [[0.28235294, 0.47843137, 0.7372549 ],
         [0.2627451 , 0.43921569, 0.70196078]

In [29]:
def gen(inputs, image_shape):
    l=32
    x=Dense((image_shape[0]//l) * (image_shape[0]//l) * 128)(inputs)
    x=Reshape((image_shape[0]//l, image_shape[0]//l, 128))(x)
    
    x=UpSampling2D(3)(x)
    x=Conv2D(filters = 256, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x=UpSampling2D(3)(x)
    x=Conv2D(filters = 128, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x=UpSampling2D(2)(x)
    x=Conv2D(filters = 64, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x=UpSampling2D(2)(x)
    x=Conv2D(filters = 32, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(alpha=0.2)(x)

#    x=Conv2DTranspose(filters = 32, kernel_size=3, strides=1, padding='same', activation = 'sigmoid')(x)
#    x=BatchNormalization()(x)
    
    x=Conv2D(filters = 3, kernel_size=3, padding='same')(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    generator=Model(inputs, x)
    return generator

In [30]:
def dis(inputs):
    x=inputs
    
    x= Conv2D(filters= 256, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x= Conv2D(filters= 128, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x= Conv2D(filters= 64, kernel_size=3, strides=2, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x= Conv2D(filters= 32, kernel_size=3, strides=1, padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)
    
    x=Flatten()(x)
    x=Dense(32, activation= 'tanh')(x)
    x=Dense(1, activation = 'softmax')(x)
    
    discriminator = Model(inputs, x)
    return discriminator

In [31]:
def models(latent_size):
    input_shape=(images.shape[1],images.shape[2],images.shape[3])
    
    inputs = Input(shape=input_shape)
    discriminator=dis(inputs)
    discriminator.compile(loss = 'mse', optimizer=RMSprop(lr= 2e-4, decay=6e-8), metrics = ['accuracy'])
    
    inputs = Input(shape=(latent_size, ))
    generator=gen(inputs, input_shape)
    
    discriminator.trainable = False
    
    adversarial=Model(inputs, discriminator(generator(inputs)))
    adversarial.compile(loss='mse', optimizer = RMSprop(lr=2e-4*0.5,decay=6e-8*0.5))
    
    models=(generator, discriminator, adversarial)
    return models

In [32]:
def train(models, epochs, batch_size, latent_size, save_interval=0):
    generator, discriminator, adversarial = models
    
    for i in range(epochs):
        rand_indices = np.random.randint(0, images.shape[0], size = batch_size)
        real = images[rand_indices]
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        fake = generator.predict(noise)
        
        x = np.concatenate((real,fake))
        
        y = np.ones(batch_size*2)
        y[batch_size:]=0.0
        y=np.reshape(y,(batch_size*2,1))
        
        history = discriminator.train_on_batch(x,y)
        
        noise = np.random.uniform(0, 1.0, size = [batch_size, latent_size])
        y=np.ones(batch_size)
        y=np.reshape(y,(batch_size,1))
        
        """aloss, aacc =""" 
        history1 = adversarial.train_on_batch(noise, y)
        
        print("Epoch : %d, [adversarial loss: %f] [discriminator loss: %f, acc: %f]"%(i+1, history1, history[0], history[1]))
        
        if save_interval!=0:
            if (i+1)%save_interval==0:
                noise = np.random.uniform(0, 1.0, size = [1, latent_size])
                n=generator.predict(noise)
                n=n[0]
                n*=255
                cv2.imwrite("%d.jpeg"%(i+1),n)
                print("Saved!")

In [ ]:
model=models(40000)
train(model, 1000, 64, 40000, 10)

Epoch : 1, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 2, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 3, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 4, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 5, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 6, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 7, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 8, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 9, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 10, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Saved!
Epoch : 11, [adversarial loss: 0.000000] [discriminator loss: 0.500000, acc: 0.500000]
Epoch : 12, [adversarial loss: 0.000000] [dis

In [ ]:
images